In [1]:
from IPython.core.display import HTML
HTML('''
<script>
var logoParent = document.getElementById("kernel_logo_widget")
var logo = document.getElementById("kernel_logo_widget").getElementsByClassName("current_kernel_logo")[0];
logo.src = "https://i.ibb.co/mD4jTGQ/itclogo.jpg";
logo.style = "display: inline; width:138px; height:40px";
logoParent.innerHTML = '<a href="https://i.ibb.co/mD4jTGQ/itclogo.jpg">' + logoParent.innerHTML + '</a>';
</script>
''')

<font size=10><b>Feature Selection - Exercise</b></font>

In this exercise, we will use the adult_data_usa.csv data, that holds information about different people.
We will try to predict wheather a person earns more than 50K a year, or less than that.

We will use different feature selection techniques to select features for the models, and compare the recieved models.

# Questions

## load the dataset and explore it LIGHTLY

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

In [3]:
# your code here
df = pd.read_csv('adult_data_usa.csv')

In [4]:
df.sample(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,income
23992,50,Local-gov,100480,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,60,>50K
19590,20,?,37932,HS-grad,9,Never-married,?,Not-in-family,White,Male,0,0,40,<=50K
4106,25,Private,210184,11th,7,Separated,Craft-repair,Own-child,White,Male,0,0,40,<=50K
10326,37,Local-gov,102936,Bachelors,13,Divorced,Prof-specialty,Unmarried,White,Female,0,0,55,<=50K
12427,50,Private,189107,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,<=50K


In [5]:
df.shape

(29170, 14)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29170 entries, 0 to 29169
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             29170 non-null  int64 
 1   workclass       29170 non-null  object
 2   fnlwgt          29170 non-null  int64 
 3   education       29170 non-null  object
 4   education-num   29170 non-null  int64 
 5   marital-status  29170 non-null  object
 6   occupation      29170 non-null  object
 7   relationship    29170 non-null  object
 8   race            29170 non-null  object
 9   sex             29170 non-null  object
 10  capital-gain    29170 non-null  int64 
 11  capital-loss    29170 non-null  int64 
 12  hours-per-week  29170 non-null  int64 
 13  income          29170 non-null  object
dtypes: int64(6), object(8)
memory usage: 3.1+ MB


In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,29170.0,38.655674,13.722408,17.0,28.00,37.0,48.0,90.0
fnlwgt,29170.0,187069.021152,104877.981749,12285.0,115895.25,176730.0,234138.5,1484705.0
education-num,29170.0,10.169455,2.394966,1.0,9.00,10.0,12.0,16.0
capital-gain,29170.0,1089.229928,7381.898528,0.0,0.00,0.0,0.0,99999.0
capital-loss,29170.0,88.510593,405.681709,0.0,0.00,0.0,0.0,4356.0
hours-per-week,29170.0,40.447755,12.417203,1.0,40.00,40.0,45.0,99.0


In [8]:
df.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
income            0
dtype: int64

## Randomly split the data into train and test sets (no need for validation at this point)

In [9]:
# your code here
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns = ['income']), df.income, test_size = 0.2, random_state = 42)

In [10]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((23336, 13), (23336,), (5834, 13), (5834,))

## Encode all categorical features, and turn the dependent variable into a numeric feature as well

In [11]:
# your code here
X_train = pd.get_dummies(X_train, columns = ['workclass','education','marital-status','occupation','relationship','race','sex'])
X_test = pd.get_dummies(X_test, columns = ['workclass','education','marital-status','occupation','relationship','race','sex'])

In [12]:
X_train.shape

(23336, 66)

In [13]:
for col in (set(X_train.columns)-set(X_test.columns)):
  X_test[col] = 0

for col in (set(X_test.columns)-set(X_train.columns)):
  del X_test[col]


In [14]:
threshold = 0.8
new_X_train = X_train.drop(columns = ['age','fnlwgt','hours-per-week','capital-gain','capital-loss','education-num'])

for col in new_X_train.columns:
  for i in range (2):
    if new_X_train[col].value_counts()[i]/new_X_train.shape[0] > threshold:
      del X_train[col]
      del X_test[col]
      break




In [15]:
X_test.columns = X_train.columns

In [16]:
X_train.shape,X_test.shape

((23336, 15), (5834, 15))

In [17]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

## Train a model of your choice, and evaluate it. What are your recall and precision?

In [18]:
# your code here
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)


In [19]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.92      0.86      4403
           1       0.56      0.33      0.41      1431

    accuracy                           0.77      5834
   macro avg       0.69      0.62      0.64      5834
weighted avg       0.75      0.77      0.75      5834



For class 0, we get a precision of 0.81 and a recall of 0.92

For class 1, we get a precision of 0.56 and a recall of 0.33

## Try to select features with the Chi-square method, and one other filter method. Indicate which features were eliminated. Train the same model with the new subset of features and report your precision and recall

#### Chi-square method

In [20]:
# your code here
X_train_cat = X_train.drop(columns = ['age','fnlwgt','education-num','hours-per-week'])
kbest = SelectKBest(score_func = chi2, k = 'all')
kbest.fit(X_train_cat,y_train)
df_useless_feat = pd.DataFrame({'chi_2_score':kbest.scores_,'p_value':kbest.pvalues_},
                               index = X_train_cat.columns)


In [21]:
useless_features = df_useless_feat[df_useless_feat.p_value >=0.05].index
useless_features

Index([], dtype='object')

None of the features have been dropped

#### f-regression method

In [22]:
X_train_num = X_train[['age','fnlwgt','education-num','hours-per-week']]
f_stat, p_value = f_regression(X_train_num,y_train)
df_useless_num = p_values_adult = pd.DataFrame({'corr': X_train_num.corrwith(pd.Series(y_train)).abs(),
                               'f_regression':f_stat,
                               'p_value':p_value},
                              index = X_train_num.columns)

In [23]:
useless_num_features = df_useless_num[df_useless_num.p_value >= 0.05].index
useless_num_features

Index(['fnlwgt'], dtype='object')

In [24]:
for col in useless_num_features:
  del X_train[col]
  del X_test[col]

'fnlwgt' has been dropped

In [25]:
X_train.shape

(23336, 14)

In [26]:
knn2 = KNeighborsClassifier()
knn2.fit(X_train,y_train)
y_pred2 = knn2.predict(X_test)

print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89      4403
           1       0.69      0.61      0.65      1431

    accuracy                           0.84      5834
   macro avg       0.78      0.76      0.77      5834
weighted avg       0.83      0.84      0.83      5834



We get much better result than before.

We get for class 0 a precision of 0.88 and a recall of 0.91

We get for class 1 a precision of 0.69 and a recall of 0.61

## Try to select features with ONE wrapper method. Indicate which features were eliminated. Train the same model with the new subset of features and report your precision and recall

**Indicate how many features you kept, and PROVIDE CONCLUSIONS**

In [27]:
# your code here
knn3 = KNeighborsClassifier(n_jobs = -1)
knn3.fit(X_train,y_train)


KNeighborsClassifier(n_jobs=-1)

In [28]:
sfs = SequentialFeatureSelector(knn3,n_features_to_select = 0.7, direction = 'forward')
sfs.fit(X_train,y_train)


SequentialFeatureSelector(estimator=KNeighborsClassifier(n_jobs=-1),
                          n_features_to_select=0.7)

In [29]:
columns_left = list(X_train.columns[sfs.get_support()])
columns_removed = list(X_train.columns[~sfs.get_support()])

In [30]:
columns_left

['education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'education_ HS-grad',
 'marital-status_ Married-civ-spouse',
 'marital-status_ Never-married',
 'relationship_ Husband',
 'sex_ Female']

Left features

'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'education_ HS-grad',
 'marital-status_ Married-civ-spouse',
 'marital-status_ Never-married',
 'relationship_ Husband',
 'sex_ Female'

In [31]:
columns_removed

['age',
 'workclass_ Private',
 'education_ Some-college',
 'relationship_ Not-in-family',
 'sex_ Male']

Removed features

'age',
 'workclass_ Private',
 'education_ Some-college',
 'relationship_ Not-in-family',
 'sex_ Male'

In [32]:
X_train = X_train[columns_left]
X_test = X_test[columns_left]

In [33]:
knn4 = KNeighborsClassifier(n_jobs = -1)
knn4.fit(X_train,y_train)
y_pred_sfs = knn4.predict(X_test)

print(classification_report(y_test,y_pred_sfs))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      4403
           1       0.75      0.59      0.66      1431

    accuracy                           0.85      5834
   macro avg       0.81      0.76      0.78      5834
weighted avg       0.85      0.85      0.85      5834



Class 0 : we get a precision of 0.88 and a recall of 0.94

Class 1 : we get a precision of 0.75 and a recall of 0.59

## Please sumarize your results. Did things happened as you expected? Please elaborate.

- By just removing redundant features, we got

class 0 : precision = 0.81 , recall = 0.92  

class 1 : precision = 0.56 , recall = 0.33

- Then by using chi-square method and f-regression method, we got

class 0 : precision = 0.88 , recall = 0.91

class 1 : precision = 0.69 , recall = 0.61

- Then by using forward selection, we got

class 0 : precision = 0.88 , recall = 0.94

class 1 : precision = 0.75 , recall = 0.59

We see that the more we work on feature selection the better our results are. By keeping only features that are dependant to the target variable, we get better result and then by keeping only the most relevant features, we get better result. It makes sense. I would say that things happened as expected.

